# scikit-learnのトレーニング♨

## 環境準備、共通的データ処理 編

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
- [共通的データ処理](#共通的データ処理)
  - [生成](#生成)
  - [加工](#加工)
  - [理解](#理解)
  - [準備](#準備)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python#y4054d35

## 環境準備

### インストール

In [ ]:
!pip install scikit-learn # scikit-learn
!pip install openpyxl # Excel読込用
!pip install seaborn # 散布図表示用
!pip install mlxtend # 分類可視化用

### インポート

#### 基本的なライブラリ

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### scikit-learn

##### 前処理

In [ ]:
from sklearn import datasets                                 # サンプル・データセット
from sklearn.datasets import make_regression                 # 回帰データセット作成
from sklearn.datasets import make_blobs                      # 分類データセット作成
from sklearn.preprocessing import StandardScaler             # 標準化
from sklearn.model_selection import train_test_split         # データ分割

##### モデル

In [ ]:
from sklearn.linear_model import LinearRegression            # 線形回帰
from sklearn.preprocessing import PolynomialFeatures         # 多項式回帰の変数変換
from sklearn.linear_model import Ridge                       # 多項式回帰のRidge回帰
from sklearn.linear_model import Lasso                       # 多項式回帰のLASSO回帰
from sklearn.linear_model import ElasticNet                  # 多項式回帰のLASSO回帰
from sklearn.linear_model import Perceptron                  # 単純パーセプトロン線形分類器
from sklearn.linear_model import LogisticRegression          # ロジスティク回帰
from sklearn.svm import SVC                                  # サポートベクターマシン（SVM）分類器
from sklearn.tree import DecisionTreeClassifier              # 決定木（分類木）
from sklearn.ensemble import RandomForestClassifier          # ランダムフォレスト（分類木）
from sklearn.ensemble import GradientBoostingClassifier      # 勾配ブースティング木（分類木）
from sklearn.decomposition import PCA                        # 主成分分析
from sklearn.cluster import KMeans                           # k-means法 クラスタ分析
from sklearn.feature_extraction.text import CountVectorizer  # 自然言語処理ベクトル化
from sklearn.feature_extraction.text import TfidfTransformer # 自然言語ベクトルのTF-IDF計算
from sklearn.decomposition import LatentDirichletAllocation  # 自然言語ベクトルからLDAトピック抽出

##### 精度評価

In [ ]:
from sklearn import metrics                                  # モデル評価
from sklearn.metrics import mean_squared_error as mse        # 精度評価（mse）
from sklearn.metrics import confusion_matrix                 # 混同行列
from sklearn.metrics import silhouette_samples               # シルエット係数
from sklearn.model_selection import cross_val_score          # 交差検証法
from sklearn.model_selection import KFold                    # k分割交差検証法
from sklearn.model_selection import StratifiedKFold          # 層化交差検証法
from sklearn.model_selection import GridSearchCV             # グリッドサーチ

#### その他

##### 可視化

In [ ]:
import seaborn as sns                                       # matplotlibラッパ
from mlxtend.plotting import plot_decision_regions          # 決定領域表示関数
from matplotlib import cm                                   # カラーマップ処理

##### その他

In [ ]:
from numpy import linalg as LA # 線形代数ライブラリ

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### jupyter上で利用するとき

In [ ]:
%matplotlib inline

## 共通的データ処理
- 以下の共通的な処理では、何らかのデータをロードしてから試す。
- 赤ワインの品質：回帰または分類モデリングのためのシンプルでクリーンな実践用データセット

### 生成

#### テストデータの作成

##### 回帰データセット

###### 生成

In [ ]:
x, y, coef = make_regression(random_state=12, 
                       n_samples=100,   # サンプル数 100
                       n_features=4,    # 特徴量の数 4
                       n_informative=2, # 目的変数に相関の強い特徴量の数 2
                       noise=10.0,      # ノイズ 10.0
                       bias=-0.0,
                       coef=True)

In [ ]:
# 相関係数
coef

###### 表示

In [ ]:
df_x=pd.DataFrame(x,columns=['a','b','c','d'])
df_y=pd.DataFrame(y,columns=['y'])
df=pd.concat([df_x, df_y],axis=1)

sns.pairplot(df)
plt.show()

##### 分類データセット

###### 生成

In [ ]:
x, y = make_blobs(random_state=8,
                  n_samples=100,   # サンプル数 100
                  n_features=2,    # 特徴量の数を 2
                  cluster_std=1.5, # 標準偏差
                  centers=3)       # 塊数を3

###### 表示

In [ ]:
# 関数
def cluster_plot(n_clusters, x, y):
    plt.figure()
    for target, marker, color in zip(range(3), '>ox', 'rgb'): # 3値分類
        # y==targetで、boolのnumpy.ndarrayベクトルが返るのでコレで行を指定している。
        plt.scatter(x[y==target, 0], x[y==target, 1], marker=marker, color=color)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()

# 散布
cluster_plot(3, x, y)

In [ ]:
# sns.pairplotで
df1=pd.DataFrame(x,columns=['x1','x2'])
df2=pd.DataFrame(y,columns=['y'])
df=pd.concat([df1,df2],axis=1)
sns.pairplot(df, hue='y')
plt.show()

##### 関数で近似できるデータセット

###### 適当な関数を定義

In [ ]:
def GenerateData0(x):
    y = 0.0001 * (x**3 + x**2 + x + 1)
    return y

###### 関数上のプロット

In [ ]:
# xの範囲を指定
x_plot = np.arange(-25, 25, 0.1)
# yを関数で指定
y_plot = GenerateData0(x_plot)
# 機械学習用に変換
x_plot = x_plot.reshape(-1, 1)

###### サンプルの生成

In [ ]:
# 乱数生成器の状態を指定
np.random.seed(3)
# 正規分布に従ってX個のデータ点を生成
X = 30
x = np.random.normal(0, 10, X)
# 対応するyを関数で生成
y = GenerateData0(x)
# 正規分布に従うノイズを加える
y += np.random.normal(0, 0.25, len(y))
# 機械学習用に変換
x = x.reshape(-1, 1)

###### 関数の描画と散布図で可視化

In [ ]:
# 関数を描画
plt.plot(x_plot, y_plot, color='gray')
# サンプルを散布
plt.scatter(x, y)
# グラフを表示
plt.show()

#### 以下の手順で使用するデータのロード（Webから
必要に応じて[環境変数にプロキシ設定](PythonTraining.ipynb)をしておくと良い。

In [ ]:
proxies = { # プロキシ設定
"http":"http://<user_name>:<password>@<proxy_host>:<proxy_port>/",
"https":"https://<user_name>:<password>@<proxy_host>:<proxy_port>/"
}

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ
df = pd.read_csv(io.BytesIO(res.content), encoding='shift-jis', sep=";")
df

### 加工
切出や結合など。

### 理解

#### 基本統計量
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90#j6ea2557

In [ ]:
df.describe()

#### 相関
目的変数と相関の強い説明変数を調査。

##### 相関係数（相関行列）
コレだと数値の和列で解り難い。  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E7%B5%B1%E8%A8%88%E8%A7%A3%E6%9E%90#bad01586

In [ ]:
df.corr()

##### ヒートマップに表示
ヒートマップにすると相関の強弱が解り易い。

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df.corr(), annot=True, square=True, fmt='.2f')
plt.show()

##### 散布図行列

###### 通常

In [ ]:
# 列数が多いとハングするので注意。
#sns.pairplot(df)
#plt.show()

###### 不要な列を削除して

In [ ]:
# 列名の確認
df.columns

In [ ]:
#df2=df.drop(['削除する','列名の','リスト'], axis=1)
df2=df.drop(['volatile acidity','citric acid','residual sugar','chlorides',
             'free sulfur dioxide','total sulfur dioxide','sulphates'], axis=1)

###### 散布図行列を表示

In [ ]:
sns.pairplot(df2)
plt.show()

###### カテゴリ分類した散布図行列を表示

In [ ]:
sns.pairplot(df2, hue='quality')
plt.show()

###### 散布図行列で特徴量を選択
- 回帰の場合、目的変数と相関関係が高い説明変数を選択する。
- 分類の場合、目的変数を空間上で分離できそうな説明変数を選択する。
- 相関関係が高い説明変数同士は問題を起こす可能性（マルチコ）。

##### 欠損率の計算と確認

In [ ]:
df.isnull().sum() / len(df) 

### 準備

#### [データのクリーニング](PandasTraining.ipynb)
- リストワイズ法（削除）
- ペアワイズ法（削除）
- 補完値で置換

#### [データの構築・統合](PandasTraining.ipynb)
- 単一属性変換
  - カテゴリ → 数値
  - 数値 → カテゴリ
- One-Hotエンコーディング
  - [NumPyの場合、to_categoricalを使う。](NumPyTraining3.ipynb)
  - [Pandasの場合、get_dummiesを使う。](PandasTraining.ipynb)（基本コッチ

#### （PG上ある）
CRIPS-DM上には明記されていないが、PG的に必要。

##### 説明変数・目的変数の選択・分割

##### 取り敢えず以下のサンプルを使う。

###### DFでやるか

In [ ]:
df_x = df2.loc[:, ['alcohol']]
df_y = df2.loc[:, ['quality']]
x1 = np.array(df_x)
y1 = np.array(df_y)

###### NPでやるか

In [ ]:
xy = np.array(df2)
xy.shape

In [ ]:
x2 = xy[:, 3:4]
y2 = xy[:, 4:5]

###### 比較してチェック

In [ ]:
print(np.allclose(x1, x2))
print(np.allclose(y1, y2))

In [ ]:
x=x1
y=y1

##### 標準化・正規化
DFでやる方法とNPでやる方法があるが、
[説明変数と目的変数の切出](#説明変数と目的変数の切出)基本NPでやる。

In [ ]:
# 平均
print(x.mean())

In [ ]:
# 標準偏差
print(x.std())

###### 標準化

In [ ]:
ss = StandardScaler()
# 引数は numpy.ndarray
sx = ss.fit_transform(x)

In [ ]:
# 平均が ≒ 0
print(sx.mean())

In [ ]:
# 標準偏差が ≒ 1
print(sx.std())

###### 正規化

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(0, 1))
# 引数は numpy.ndarray
mmsx = mms.fit_transform(x)

In [ ]:
# 平均が0-1の間に。
print(mmsx.mean())

In [ ]:
# 標準偏差は
print(mmsx.std())

##### 学習・テストデータの分割

###### ホールド・アウト法
- test_size = 0.3  
train:test = 7:3 で分割。
- random_state = 0  
毎回同じサンプルに分割

In [ ]:
# 学習・テストデータの分割（ホールド・アウト法
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

# データ分割の結果確認
# (numpy.ndarray).shapeメソッドで確認できる。
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

###### [交差検証法](ScikitLearnTraining5.ipynb)